In [ ]:
# Import class
import os
import pickle
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import NBAStats
stats_file = os.path.join(parent_dir, 'Data', 'nba_stats.pickle')
print(stats_file)
nba_stats = pickle.load(open(stats_file, 'rb'))

import player_market_class
import scraper_SB
import scraper_LB
import empirical
import pandas as pd
import datetime as dt

# Motivation
Being able to see the best prices across multiple exchanges.
* Find arbs
* Save manual search

# Plan
* Get lists of PlayerMarket class items
* Write a function to return a consolidated list of the best odds

In [ ]:
market_date = dt.date(2019, 1, 18)
market_date_LB = market_date.strftime("%d/%m/%Y")
market_date_SB = market_date.strftime("%d/%m/%y")

links_LB = scraper_LB.get_links_LB(market_date_LB)
links_SB = scraper_SB.get_links_SB(market_date_SB)


In [ ]:
odds_str_LB = []
for link in links_LB:
    odds_str_LB.extend(scraper_LB.get_player_markets_LB(link))

In [ ]:
odds_str_SB = []
for link in links_SB:
    odds_str_SB.extend(scraper_SB.get_player_markets_SB(link))

In [ ]:
odds_str_LB.remove('Nicolas Batum Total Assists')

In [ ]:
player_markets_LB = [player_market_class.PlayerMarketLadbrokes(odds_str) for odds_str in odds_str_LB]
player_markets_SB = [player_market_class.PlayerMarketSportsBet(odds_str) for odds_str in odds_str_SB]

market_names_LB = [player_market.get_market_name() for player_market in player_markets_LB]
market_names_SB = [player_market.get_market_name() for player_market in player_markets_SB]

common_market_names = list(set(market_names_LB).intersection(set(market_names_SB)))

In [ ]:
best_markets = []
for market in common_market_names:
    i_LB = market_names_LB.index(market)
    i_SB = market_names_SB.index(market)
    
    p_market_LB = player_markets_LB[i_LB]
    p_market_SB = player_markets_SB[i_SB]
    
    best_market = p_market_LB.combine_odds(p_market_SB)
    best_markets.append(best_market)

for market in player_markets_LB:
    if market.get_market_name() in common_market_names:
        continue
    else:
        best_markets.append(market)

for market in player_markets_SB:
    if market.get_market_name() in common_market_names:
        continue
    else:
        best_markets.append(market)    

In [ ]:
for market in best_markets:
    print(market, end='\n\n')

## Arb finder

In [ ]:
bookie_pr_edge_list = []
for market in best_markets:
    odds_over = market.get_odds_over()
    odds_under = market.get_odds_under()
    
    bookie_pr_edge = 1/float(odds_over) + 1/float(odds_under) - 1
    bookie_pr_edge_list.append(bookie_pr_edge)

for edge, market in sorted(zip(bookie_pr_edge_list, best_markets), key=lambda t:t[0]):
    print('Bookie edge {:.2f}'.format(edge))
    print(market, end='\n\n')
    